<a href="https://colab.research.google.com/github/misanchz98/bitcoin-direction-prediction/blob/main/03_modeling/03_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Modelos de Deep Learning para Series Temporales con Walk-Forward Validation

En este notebook implementamos diferentes modelos de **Deep Learning** y técnicas de validación temporal con **Purged Walk-Forward Split**.  

Incluye:
- Preprocesamiento y creación de secuencias.
- Modelos: LSTM, GRU, CNN-LSTM, Transformer y TCN.
- Métricas personalizadas y evaluación.
- Importancia de características.
- Pipeline de entrenamiento y validación.

## 🔹 1. Librerías
Instalamos e importamos las librerías necesarias para manipulación de datos, visualización, machine learning y deep learning.


In [3]:
# =============================================================================
# LIBRERIAS
# =============================================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import warnings

# Establecer el nivel de advertencias a "ignore" para ignorar todas las advertencias
warnings.filterwarnings("ignore")

In [22]:
# 1️⃣ Clonar el repositorio desde GitHub
!git clone https://github.com/misanchz98/bitcoin-direction-prediction.git


# 2️⃣ Importar librerías necesariason

# 3️⃣ Ruta al archivo pipelines.py
# Ajusta esta ruta según tu repositorio
#pipeline_path = '/content/bitcoin-direction-prediction/03_modeling/utils/pipelines.py'
#
## 4️⃣ Cargar el módulo dinámicamente
#spec = importlib.util.spec_from_file_location("pipelines", pipeline_path)
#pipelines = importlib.util.module_from_spec(spec)
#spec.loader.exec_module(pipelines)
#
## 5️⃣ Asignar la función a una variable
#run_pipeline_random_search = pipelines.run_pipeline_random_search
#print("Pipeline importado correctamente. ¡Listo para usar!")
#

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'bitcoin-direction-prediction': No such file or directory


In [11]:
import shutil
#shutil.rmtree('/content/utils')

## 🔹 2. Semillas
Para garantizar **reproducibilidad** en los experimentos, es importante fijar las semillas de las librerías que generan números aleatorios:

- `os.environ['PYTHONHASHSEED']` → controla el hash en Python.  
- `numpy.random.seed` → asegura resultados reproducibles en operaciones de NumPy.  
- `random.seed` → fija la semilla del generador de números aleatorios nativo de Python.  
- `tf.random.set_seed` → fija la semilla para TensorFlow y Keras.  

Esto ayuda a que los modelos se entrenen con resultados consistentes entre ejecuciones.

In [ ]:
# resetting the seeds for reproducibility
def reset_random_seeds():
    n = 42
    os.environ['PYTHONHASHSEED'] = str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    random.seed(n)

reset_random_seeds()

## 🔹 3. Conjunto de Datos
Importamos el conjunto de datos en nuestro entorno de trabajo. Se encuentran almacenados en un archivo CSV llamado `btc_historical_data_eda.csv`, cuya obtención se explica en el *notebook* `02_data_analysis.ipynb`.

In [ ]:
# Importamos CSV
url = 'https://raw.githubusercontent.com/misanchz98/bitcoin-direction-prediction/main/02_data_analysis/data/btc_historical_data_eda.csv'
df_bitcoin = pd.read_csv(url, parse_dates=['Open time'])
df_bitcoin

## 🔹 4. Ajuste de Hiperparámetros


Antes de realizar el ajuste de hiperparámetros de las redes neuronales:

1. Establecemos la columna `Open time` como índice del conjunto de datos.
2. Creamos la columna `Return`:

```python
df_bitcoin["Return"] = np.log(df_bitcoin["Close"] / df_bitcoin["Close"].shift(1)).fillna(0)
```

  - **¿Qué significa?**

    - `df_bitcoin["Close"]` → Precio de cierre del Bitcoin.
    - `df_bitcoin["Close"].shift(1)` → Precio de cierre del día anterior.
    - `df_bitcoin["Close"] / df_bitcoin["Close"].shift(1)` → Retorno simple diario:

  - **Fórmula del retorno simple**:
$$
\text{Retorno simple} = \frac{P_t}{P_{t-1}}
$$

    - `np.log(...)` → Retorno logarítmico, que tiene varias ventajas:
      - Es **aditivo en el tiempo**: la suma de retornos logarítmicos de varios días es igual al log del retorno total.
      - Maneja mejor la **volatilidad** y los efectos de la **capitalización continua**.

    - `.fillna(0)` → El primer día no tiene precio anterior, se rellena con 0 para evitar valores nulos.


  - **¿Para qué se usa?** En nuestro pipeline de trading:

    - `Return` no es la variable objetivo (`y`), sino información adicional.
    - Se utiliza para calcular métricas financieras, principalmente el **Sharpe ratio**.
    - Dependiendo de la estrategia (`longonly`, `longshort`, `shortonly`), se combina con las predicciones `y_pred` para simular **ganancias o pérdidas**.


In [ ]:
# Asegúrate que tu DataFrame tenga 'Target' y 'Return'

# Establecemos columna 'Open time' como indice
df_bitcoin["Open time"] = pd.to_datetime(df_bitcoin["Open time"])
df_bitcoin = df_bitcoin.set_index("Open time")

# Creamos la columna Return
df_bitcoin["Return"] = np.log(df_bitcoin["Close"] / df_bitcoin["Close"].shift(1)).fillna(0)

### Estrategia LONGONLY
- Qué hace: Solo toma posiciones largas (compra).
- Objetivo: Ganar cuando el precio sube.
- Ejemplo: Compras BTC esperando que suba, y vendes más caro.
- Ventaja: Menor riesgo que el shorting, especialmente en mercados alcistas.
- Limitación: No se beneficia de caídas del mercado.

In [ ]:
# Ejecutar la búsqueda - longonly
results = run_pipeline_random_search(
    df_bitcoin,
    target_col="Target",
    return_col="Return",
    window_size=30,
    horizon=1,
    test_size=0.2,
    strategy="longonly",   # longonly | shortonly | longshort
    scoring="sharpe",      # sharpe | accuracy | f1 | ...
    n_iter=5               # número de combinaciones a probar
)

In [ ]:
# Obtener un modelo específico
best_model_lstm_longonly = results.get_model_by_id("LSTM_comb5")
best_model_gru_longonly = results.get_model_by_id("GRU_comb4")
best_model_cnn_lstm_longonly = results.get_model_by_id("LSTM_CNN_comb1")
best_model_cnn_gru_longonly = results.get_model_by_id("GRU_CNN_comb5")

### Estrategia LONGSHORT
- Qué hace: Puede tomar ambas posiciones, largas y cortas.
- Objetivo: Aprovechar tanto subidas como bajadas.
- Ejemplo: Compras BTC y vendes ETH si crees que BTC va a subir y ETH a bajar.
- Ventaja: Más flexible, puede generar ganancias en cualquier dirección del mercado.
- Limitación: Más compleja, requiere buena gestión de riesgo.

In [ ]:
# Ejecutar la búsqueda - longshort
results = run_pipeline_random_search(
    df_bitcoin,
    target_col="Target",
    return_col="Return",
    window_size=30,
    horizon=1,
    test_size=0.2,
    strategy="longshort",   # longonly | shortonly | longshort
    scoring="sharpe",      # sharpe | accuracy | f1 | ...
    n_iter=5               # número de combinaciones a probar
)

In [ ]:
# Obtener un modelo específico
best_model_lstm_longshort = results.get_model_by_id("LSTM_comb5")
best_model_gru_longshort = results.get_model_by_id("GRU_comb5")
best_model_cnn_lstm_longshort = results.get_model_by_id("LSTM_CNN_comb3")
best_model_cnn_gru_longshort = results.get_model_by_id("GRU_CNN_comb5")